In [2]:
import multiprocessing
import pyspark
import pandas as pd
import numpy as np
np.random.seed(13)
from pyspark.sql.functions import lit
from pydataset import data
import pydataset
from pyspark.sql.functions import min,max,avg,sum,concat, lit
from pyspark.sql.functions import when
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
import pandas as pd
from pydataset import data
from vega_datasets import data as v_data
import pyspark.sql.functions as F

In [3]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [4]:
np.random.seed(456)

pandas_dataframe = pd.DataFrame(dict(n=np.arange(20)))

df = spark.createDataFrame(pandas_dataframe)

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [5]:
df = spark.createDataFrame(pd.DataFrame(['python', 'java','matlab','sql','javascript','r'], columns=['language']))
df

DataFrame[language: string]

In [6]:
df.printSchema()

root
 |-- language: string (nullable = true)



In [7]:
df.count(), len(df.columns)

(6, 1)

In [8]:
df.show(5)

+----------+
|  language|
+----------+
|    python|
|      java|
|    matlab|
|       sql|
|javascript|
+----------+
only showing top 5 rows



In [9]:

mpg = spark.createDataFrame(data('mpg'))
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [10]:
sentence = F.concat(F.lit('The '), 
                    mpg.year, F.lit(' '), 
                    mpg.manufacturer, 
                    F.lit(' '), 
                    mpg.model, 
                    F.lit(' has a '), 
                    mpg.cyl,
                    F.lit(' cylinder engine.')).alias('description')
mpg.select(sentence).show(5,truncate=False)

+-----------------------------------------+
|description                              |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 6 cylinder engine.|
+-----------------------------------------+
only showing top 5 rows



In [11]:
simple_trans = F.regexp_extract(mpg.trans, 
                                r'(^\w+?)\(.*\)', 
                                1).alias('simple_trans')
mpg.select(simple_trans).show(5)




(mpg.select(
    '*', (
        F.when(mpg.trans.like('auto%'), 'auto')
        .when(mpg.trans.like('manual%'), 'manual')
        .otherwise('unknown')
        .alias('transtype')))
    .drop('trans')).show(10)

+------------+
|simple_trans|
+------------+
|        auto|
|      manual|
|      manual|
|        auto|
|        auto|
+------------+
only showing top 5 rows

+------------+----------+-----+----+---+---+---+---+---+-------+---------+
|manufacturer|     model|displ|year|cyl|drv|cty|hwy| fl|  class|transtype|
+------------+----------+-----+----+---+---+---+---+---+-------+---------+
|        audi|        a4|  1.8|1999|  4|  f| 18| 29|  p|compact|     auto|
|        audi|        a4|  1.8|1999|  4|  f| 21| 29|  p|compact|   manual|
|        audi|        a4|  2.0|2008|  4|  f| 20| 31|  p|compact|   manual|
|        audi|        a4|  2.0|2008|  4|  f| 21| 30|  p|compact|     auto|
|        audi|        a4|  2.8|1999|  6|  f| 16| 26|  p|compact|     auto|
|        audi|        a4|  2.8|1999|  6|  f| 18| 26|  p|compact|   manual|
|        audi|        a4|  3.1|2008|  6|  f| 18| 27|  p|compact|     auto|
|        audi|a4 quattro|  1.8|1999|  4|  4| 18| 26|  p|compact|   manual|
|        audi|a

In [12]:
tips = spark.createDataFrame(data('tips'))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [13]:
smoke_or_no = F.when(tips.smoker == 'Yes', 1).otherwise(0) 
smoke_or_no
tips.select(F.mean(smoke_or_no)).show()

+-----------------------------------------------+
|avg(CASE WHEN (smoker = Yes) THEN 1 ELSE 0 END)|
+-----------------------------------------------+
|                            0.38114754098360654|
+-----------------------------------------------+



In [14]:

tip_pct = F.round((tips.tip/tips.total_bill),4).alias('tip_pct')
tips.select('total_bill', 'tip',tip_pct).show(5)

+----------+----+-------+
|total_bill| tip|tip_pct|
+----------+----+-------+
|     16.99|1.01| 0.0594|
|     10.34|1.66| 0.1605|
|     21.01| 3.5| 0.1666|
|     23.68|3.31| 0.1398|
|     24.59|3.61| 0.1468|
+----------+----+-------+
only showing top 5 rows



In [15]:
tips.select('sex','smoker',tip_pct).groupBy('smoker').pivot('sex').agg(F.mean('tip_pct')).show()

+------+-------------------+-------------------+
|smoker|             Female|               Male|
+------+-------------------+-------------------+
|    No| 0.1569111111111111| 0.1606659793814433|
|   Yes|0.18214545454545455|0.15276666666666663|
+------+-------------------+-------------------+



In [16]:
weather = v_data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [17]:
cel_to_far = lambda x: F.round((x * (9/5) + 32),2)
weather.select(cel_to_far(weather.temp_max).alias('temp_max(F)'), cel_to_far(weather.temp_min).alias('temp_min(F)')).show(5)


+-----------+-----------+
|temp_max(F)|temp_min(F)|
+-----------+-----------+
|      55.04|       41.0|
|      51.08|      37.04|
|      53.06|      44.96|
|      53.96|      42.08|
|      48.02|      37.04|
+-----------+-----------+
only showing top 5 rows



In [18]:
month = F.regexp_extract(weather.date, r'(\w+?)\-(\w\w)\-\w\w', 2).alias('month')
weather.select(month, 'precipitation')\
       .where(weather.weather== 'rain')\
       .groupBy('month')\
       .mean()\
       .show(5)

+-----+-------------------+
|month| avg(precipitation)|
+-----+-------------------+
|   07| 1.8785714285714286|
|   11|               8.42|
|   01|  6.614285714285714|
|   09|0.22499999999999998|
|   05| 3.2624999999999997|
+-----+-------------------+
only showing top 5 rows



In [19]:
year = F.regexp_extract(weather.date, r'(\w+?)\-(\w\w)\-\w\w', 1).alias('year')
weather.select(year, 'wind').groupBy('year').mean().orderBy(F.desc('avg(wind)')).show(1)

+----+-----------------+
|year|        avg(wind)|
+----+-----------------+
|2012|3.400819672131147|
+----+-----------------+
only showing top 1 row



In [20]:
weather.select(month, 'weather').where(month == '01').groupBy('weather').count().orderBy(F.desc('count')).show(1)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
+-------+-----+
only showing top 1 row



In [21]:
weather.select('temp_max','temp_min')\
       .where((month == '07')&
             ((year == '2013')|
              (year == '2014')))\
       .select(F.mean('temp_max'),F.mean('temp_min'))\
       .show()

+-----------------+------------------+
|    avg(temp_max)|     avg(temp_min)|
+-----------------+------------------+
|26.49677419354839|14.179032258064519|
+-----------------+------------------+



In [22]:
rainy_days =  F.when(weather.filter((year == '2015')&
                                   ((month == '07' )|
                                    (month == '08' )|
                                    (month == '09'))).weather == 'rain', 1)\
                            .otherwise(0)
weather.select(F.mean(rainy_days)).show()

+-------------------------------------------------+
|avg(CASE WHEN (weather = rain) THEN 1 ELSE 0 END)|
+-------------------------------------------------+
|                              0.17727583846680356|
+-------------------------------------------------+



In [24]:
rained_or_not = F.when(weather.precipitation != '0', 1).otherwise(0).alias('rained')
weather.select('*', rained_or_not).groupBy(year).agg(F.mean('rained')).show()

+----+-------------------+
|year|        avg(rained)|
+----+-------------------+
|2012|0.48360655737704916|
|2014|  0.410958904109589|
|2013|0.41643835616438357|
|2015|0.39452054794520547|
+----+-------------------+

